In [39]:
import pandas as pd
import numpy as np
import csv

In [40]:
from data import *
df = get_data_local()
df = clean_data(df)

✅ data received
✅ data cleaned


In [10]:
df

,from_bank,from_account,to_bank,to_account,receiving_currency,payment_currency,payment_format,is_laundering,amount_paid_USD,amount_received_USD,month,day,hour,minute
0,3208,8000F4580,1,8000F5340,USD,USD,Cheque,0,0.010000,0.010000,9,1,0,20
1,12,8000EC280,2439,8017BF800,USD,USD,Credit Card,0,7.660000,7.660000,9,1,0,26
2,1,8000EDEC0,211050,80AEF5310,USD,USD,Credit Card,0,383.709991,383.709991,9,1,0,21
3,1,8000F4510,11813,8011305D0,USD,USD,Credit Card,0,9.820000,9.820000,9,1,0,4
4,1,8000F4FE0,245335,812ED62E0,USD,USD,Credit Card,0,4.010000,4.010000,9,1,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4425429,54219,8148A6631,256398,8148A8711,XBT,XBT,Bitcoin,0,0.154978,0.154978,9,10,23,57
4425430,15,8148A8671,256398,8148A8711,XBT,XBT,Bitcoin,0,0.108128,0.108128,9,10,23,35
4425431,154365,8148A6771,256398,8148A8711,XBT,XBT,Bitcoin,0,0.004988,0.004988,9,10,23,52
4425432,256398,8148A6311,256398,8148A8711,XBT,XBT,Bitcoin,0,0.038417,0.038417,9,10,23,46


In [11]:
# list all unique accounts

from_accounts = df["from_account"].to_numpy()
to_accounts  = df["to_account"].to_numpy()

all_accounts = np.append(from_accounts, to_accounts)
all_accounts = np.unique(all_accounts)

In [5]:
# show only fraud

fraud_mask = df["is_laundering"] == 1

fraud_df = df[fraud_mask]

fraud_from = fraud_df["from_account"].to_numpy()
fraud_to  = fraud_df["to_account"].to_numpy()

all_fraud = np.append(fraud_from, fraud_to)
all_fraud = np.unique(all_fraud)

In [6]:
# get df with only non_fraud data 

non_fraud_mask = ~df["from_account"].isin(all_fraud) | ~df["to_account"].isin(all_fraud)

non_fraud_df = df[non_fraud_mask]

non_fraud_absolut = len(non_fraud_df)

In [7]:
expected_fraud_amount = 0.09

In [51]:
def cut_non_fraud(df, expected_fraud_amount=0.1):
    """
    This function randomly cuts non-fraud transactions to artificially increase amount of fraud
    """
    
    # list all unique accounts

    from_accounts = df["from_account"].to_numpy()
    to_accounts  = df["to_account"].to_numpy()

    all_accounts = np.append(from_accounts, to_accounts)
    all_accounts = np.unique(all_accounts)
    
    # show only fraud

    fraud_mask = df["is_laundering"] == 1

    fraud_df = df[fraud_mask]

    fraud_from = fraud_df["from_account"].to_numpy()
    fraud_to  = fraud_df["to_account"].to_numpy()

    all_fraud = np.append(fraud_from, fraud_to)
    all_fraud = np.unique(all_fraud)
    
    # keep this part for final df
    
    all_fraud_df_mask = df["from_account"].isin(all_fraud) | df["to_account"].isin(all_fraud)
    
    all_fraud_df = df[all_fraud_df_mask]
    
    # get df with only non_fraud data 

    non_fraud_mask = ~(df["from_account"].isin(all_fraud) | df["to_account"].isin(all_fraud))

    non_fraud_df = df[non_fraud_mask]
    
    
    # getting amount for non-fraud
    
    all_fraud_absolut = len(fraud_df)
    
    new_amount_total = all_fraud_absolut/expected_fraud_amount
    
    new_amount_non_fraud = int(new_amount_total - all_fraud_absolut)
    
    new_non_fraud_df = non_fraud_df.sample(n=new_amount_non_fraud)
    
    new_df = pd.concat([new_non_fraud_df, fraud_df])
    
    print(f"✅ randomly removed {len(df)-len(new_df)} non-fraudulent rows from the data frame.")
    
    return new_df
    

In [52]:
dfdf = cut_non_fraud(df, expected_fraud_amount=0.1)

✅ randomly removed 4373664 non-fraudulent rows from the data frame.
